In [ ]:
import numpy as np
import random
import pandas as pd
import unidecode
import getpass
import smtplib
from string import Template
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
def mail_formating(string):
    string = unidecode.unidecode(string)
    string = string.lower()
    return string

In [ ]:
def load_mail_template(language):
    file_name = language + '_mail_template.txt'
    with open(file_name, mode='r', encoding='utf-8') as file:
        mail = Template(file.read())
    return mail 

In [ ]:
def random_derangement(n):
    while True:
        v = np.arange(n)
        for j in range(n - 1, -1, -1):
            p = random.randint(0, j)
            if v[p] == j:
                break
            else:
                v[j], v[p] = v[p], v[j]
        else:
            if v[0] != 0:
                return v

In [ ]:
name_list = pd.read_csv("name_list.csv", header=0)

In [ ]:
name_list['Email address'] = name_list.apply(lambda row: mail_formating(row['First name']) + '.' +
                                                       mail_formating(row['Last name']) + '@lip6.fr', axis=1)

In [ ]:
random.seed(0)
der = random_derangement(len(name_list))
name_list['To'] = name_list['First name'].iloc[der].reset_index(drop=True)

In [ ]:
fr_mail_subject = '[Secret Santa] Tu dois faire un cadeau à...'
en_mail_subject = '[Secret Santa] You must give a present to...'

In [ ]:
fr_mail = load_mail_template('fr')
en_mail = load_mail_template('en')

In [ ]:
user_email = input('Enter your email address: ')
user_password = getpass.getpass("Enter your e-mail password: ")

In [ ]:
try:
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.ehlo()
    server.starttls()
    server.login(user_email, user_password)
    print("Successfully connected to server")
except:
    print("Failed to connect to server")    

In [ ]:
for index, row in name_list.iterrows():
    #if row['First name'] != 'Marvin':
        #continue
    msg = MIMEMultipart() # create a message
    
    # setup the parameters of the message
    msg['From'] = user_email
    msg['To'] = row['Email address']
    
    # add in the actual person name to the message template
    if row['Language'] == 'en':
        custom_mail = en_mail.substitute(SANTA_NAME=row['First name'], RECIPIENT_NAME=row['To'])
        msg['Subject'] = en_mail_subject
    elif row['Language'] == 'fr':
        custom_mail = fr_mail.substitute(SANTA_NAME=row['First name'], RECIPIENT_NAME=row['To'])
        msg['Subject'] = fr_mail_subject
        
    print(msg)
    
    # add in the message body
    msg.attach(MIMEText(custom_mail, 'plain'))
        
    # send the message via the server set up earlier.
    #server.send_message(msg)
    print("Send mail to ", row['First name'], ' ', row['Last name'])
    del msg

In [ ]:
try:
    server.close()
    print("Successfuly disconnected from server")
except:
    print("Failed to disconnect from server")